# GLM-4-9B-Chat微调实战-LoRA技术微调

## 步骤1 导入相关包

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

# 模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('ZhipuAI/chatglm3-6b-base')

# 设置关键参数
dataset_name_or_path = "llm-wizard/alpaca-gpt4-data-zh"
model_name_or_path = "/root/autodl-tmp/cache/modelscope/hub/ZhipuAI/chatglm3-6b-base"
train_output_dir = "/root/autodl-tmp/cache/finetuning/chatglm3-6b-base-lora"

/root/autodl-tmp/conda/envs/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-03 10:01:39,486 - modelscope - WARNING - Model revision not specified, use revision: v1.0.1


## 步骤2 加载数据集

In [2]:
ds = load_dataset(dataset_name_or_path,split='train')
ds

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 48818
})

In [3]:
ds[:1]

{'instruction': ['保持健康的三个提示。'],
 'input': [''],
 'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。']}

## 步骤3 数据集预处理

In [4]:
#加载本地模型，提前下载到本地
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
tokenizer

ChatGLMTokenizer(name_or_path='/root/autodl-tmp/cache/modelscope/hub/ZhipuAI/chatglm3-6b-base', vocab_size=64798, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	
}

In [5]:
def process_func(example):
    MAX_LENGTH = 256 # 设置最大长度为256
    input_ids, attention_mask, labels = [], [], [] # 初始化输入ID、注意力掩码和标签列表
    instruction = "\n".join([example["instruction"], example["input"]]).strip()     # prompt
    instruction = tokenizer.build_chat_input(instruction, history=[], role="user")  # [gMASK]sop<|user|> \n prompt <|assistant|>
    response = tokenizer("\n" + example["output"], add_special_tokens=False)        # \n response
    input_ids = instruction["input_ids"][0].numpy().tolist() + response["input_ids"] + [tokenizer.eos_token_id] #eos token
    attention_mask = instruction["attention_mask"][0].numpy().tolist() + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"][0].numpy().tolist()) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 48818
})

In [7]:
# 检查input_ids的格式
tokenizer.decode(tokenized_ds[1]["input_ids"])

'[gMASK]sop<|user|> \n 三原色是什么？<|assistant|> \n三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在以发光为基础的显示设备中（如电视、计算机显示器、智能手机和平板电脑显示屏）, 三原色可混合产生大量色彩。其中红色和绿色可以混合生成黄色，红色和蓝色可以混合生成品红色，蓝色和绿色可以混合生成青色。当红色、绿色和蓝色按相等比例混合时，可以产生白色或灰色。\n\n此外，在印刷和绘画中，三原色指的是以颜料为基础的红、黄和蓝颜色（RYB）。这三种颜色用以通过减色混合原理来创建色彩。不过，三原色的具体定义并不唯一，不同的颜色系统可能会采用不同的三原色。'

In [8]:
# 检查labels数据格式
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'\n三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在以发光为基础的显示设备中（如电视、计算机显示器、智能手机和平板电脑显示屏）, 三原色可混合产生大量色彩。其中红色和绿色可以混合生成黄色，红色和蓝色可以混合生成品红色，蓝色和绿色可以混合生成青色。当红色、绿色和蓝色按相等比例混合时，可以产生白色或灰色。\n\n此外，在印刷和绘画中，三原色指的是以颜料为基础的红、黄和蓝颜色（RYB）。这三种颜色用以通过减色混合原理来创建色彩。不过，三原色的具体定义并不唯一，不同的颜色系统可能会采用不同的三原色。'

## 步骤4 创建模型

In [9]:
import torch
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, 
                                             low_cpu_mem_usage=True,
                                             torch_dtype=torch.half,
                                             device_map="auto",
                                             trust_remote_code=True
                                            )
model.dtype

Loading checkpoint shards: 100%|██████████| 7/7 [00:05<00:00,  1.39it/s]


torch.float16

In [10]:
for name, param in model.named_parameters():
    print(name)

transformer.embedding.word_embeddings.weight
transformer.encoder.layers.0.input_layernorm.weight
transformer.encoder.layers.0.self_attention.query_key_value.weight
transformer.encoder.layers.0.self_attention.query_key_value.bias
transformer.encoder.layers.0.self_attention.dense.weight
transformer.encoder.layers.0.post_attention_layernorm.weight
transformer.encoder.layers.0.mlp.dense_h_to_4h.weight
transformer.encoder.layers.0.mlp.dense_4h_to_h.weight
transformer.encoder.layers.1.input_layernorm.weight
transformer.encoder.layers.1.self_attention.query_key_value.weight
transformer.encoder.layers.1.self_attention.query_key_value.bias
transformer.encoder.layers.1.self_attention.dense.weight
transformer.encoder.layers.1.post_attention_layernorm.weight
transformer.encoder.layers.1.mlp.dense_h_to_4h.weight
transformer.encoder.layers.1.mlp.dense_4h_to_h.weight
transformer.encoder.layers.2.input_layernorm.weight
transformer.encoder.layers.2.self_attention.query_key_value.weight
transformer.enco

### 1、PEFT 步骤1 配置文件

In [11]:
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

config = LoraConfig(target_modules=["query_key_value"])
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=8, target_modules={'query_key_value'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

### 2、PEFT 步骤2 创建模型

In [12]:
model = get_peft_model(model, config)
model

PeftModel(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (embedding): Embedding(
          (word_embeddings): Embedding(65024, 4096)
        )
        (rotary_pos_emb): RotaryEmbedding()
        (encoder): GLMTransformer(
          (layers): ModuleList(
            (0-27): 28 x GLMBlock(
              (input_layernorm): RMSNorm()
              (self_attention): SelfAttention(
                (query_key_value): lora.Linear(
                  (base_layer): Linear(in_features=4096, out_features=4608, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Identity()
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=4608, bias=False)
                  )
                  (l

In [13]:
# 打印出模型中可训练参数的数量
model.print_trainable_parameters()

trainable params: 1,949,696 || all params: 6,245,533,696 || trainable%: 0.0312


In [14]:
# 查看模型参数，查看LoRA层添加到哪
for name, param in model.named_parameters():
    print(name, param.shape, param.dtype)

base_model.model.transformer.embedding.word_embeddings.weight torch.Size([65024, 4096]) torch.float16
base_model.model.transformer.encoder.layers.0.input_layernorm.weight torch.Size([4096]) torch.float16
base_model.model.transformer.encoder.layers.0.self_attention.query_key_value.base_layer.weight torch.Size([4608, 4096]) torch.float16
base_model.model.transformer.encoder.layers.0.self_attention.query_key_value.base_layer.bias torch.Size([4608]) torch.float16
base_model.model.transformer.encoder.layers.0.self_attention.query_key_value.lora_A.default.weight torch.Size([8, 4096]) torch.float32
base_model.model.transformer.encoder.layers.0.self_attention.query_key_value.lora_B.default.weight torch.Size([4608, 8]) torch.float32
base_model.model.transformer.encoder.layers.0.self_attention.dense.weight torch.Size([4096, 4096]) torch.float16
base_model.model.transformer.encoder.layers.0.post_attention_layernorm.weight torch.Size([4096]) torch.float16
base_model.model.transformer.encoder.layer

## 步骤5 配置训练参数

In [15]:
args = TrainingArguments(
    output_dir=train_output_dir, #输出目录，用于存储模型和日志文件。
    per_device_train_batch_size=6, # 每个设备的训练批次大小，即每个设备每次处理的数据量，批次越大，训练时需要资源越多
    gradient_accumulation_steps=16, # 指定梯度累积步数。用于控制梯度更新的频率。在每个累积步中，模型会计算多个批次的梯度，然后一次性更新权重。这可以减少内存占用并提高训练速度。在本例子中，每16个步骤进行一次梯度更新。
    logging_steps=10, #日志记录步数，用于控制每隔多少步记录一次训练日志。
    num_train_epochs=1, #训练轮数，即模型在整个训练集上进行迭代的次数。正常情况会训练很多轮
    learning_rate=1e-4, #学习率，控制模型参数更新的速度。较小的学习率会使模型收敛得更快，但可能需要更多的训练轮数
    adam_epsilon=1e-4, #Adam优化器的epsilon值，用于防止除以零的情况。
    remove_unused_columns=False, #是否移除未使用的列，如果设置为False，则保留所有列，否则只保留模型所需的列。
    max_steps=100  # 为节省时间，设置最大步数为 100
)

## 步骤6 创建训练器

In [16]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

max_steps is given, it will override any value given in num_train_epochs


## 步骤7 模型训练

In [17]:
trainer.train()

[2024-09-03 10:02:33,488] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Step,Training Loss
10,1.613900
20,1.648300
30,1.611100
40,1.594400
50,1.579700
60,1.585000
70,1.586900
80,1.558600
90,1.551800
100,1.543400


TrainOutput(global_step=100, training_loss=1.587314453125, metrics={'train_runtime': 1643.088, 'train_samples_per_second': 5.843, 'train_steps_per_second': 0.061, 'total_flos': 8.422256362320691e+16, 'train_loss': 1.587314453125, 'epoch': 0.19663266560157305})

```bash
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:31:00.0 Off |                  Off |
| 35%   62C    P2             297W / 450W |  19917MiB / 24564MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
+---------------------------------------------------------------------------------------+
```

## 步骤8 模型推理

In [ ]:
# 保存模型及训练状态
trainer.save_model(train_output_dir)
# 保持模型处于评估状态
model.eval()

In [ ]:
# 模型推理
model.chat(tokenizer, "如何写简历？", history=[])[0]